### Required Libraries

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import TokenTextSplitter

### Defining the models

In [6]:
Model = "llama3"; 
Embedding_Model = "mxbai-embed-large"
model = Ollama(model=Model)

### Loading the vectorstore

In [2]:
vectors = FAISS.load_local("../VectorStores/SinglePrinter", OllamaEmbeddings(model =  "mxbai-embed-large"), allow_dangerous_deserialization= True)
retriever = vectors.as_retriever()

### Defining the RAG Model

In [9]:
# A function to combine retrieved documents into a string 

def combineDocs(docs):
    context = "\n\n".join(f'Document [ Metadata : \n{doc.metadata} \nPage Content : \n{doc.page_content} ]' for doc in docs)
    return context

def combineDocs2(docs):
    context = "\n\n".join(f'{doc.page_content}'  for doc in docs)
    return context

#### Method 1 (Complicated internet answer)

In [10]:
def create_model(vectorDB):
    
    # This is where the conversation will be stored
    memory = ConversationBufferMemory()

    # Creating a conversation model for ConvChain

    def rag_retrieval(question, memory):

        # Retrieve docs from vector DB
        docs = vectorDB.similarity_search(question, k = 4)
        context = combineDocs2(docs)

        conversation = memory.load_memory_variables({}).get('history', '')

        # Add context to the prompt
        prompt = f'Answer the question based on the context below and the previous conversations. If you cannot\nanswer the question, reply "Oof that\'s a tough one, i don\'t really know this"\n\nPrevious Conversation : \n{conversation}\n\nContext : \n{context}\n\nQuestion from user: {question}'

        response = model.invoke(prompt)

        memory.add_user_message(question)
        memory.add_bot_message(response)

        return response
    
    chain = ConversationChain(memory = memory, conversation_model = rag_retrieval)

    return chain

#### Method 2 (Self)

In [15]:
chatHistory = "\n"

latestPrompt = ""

template = """
You are a helpful assistant. You are given some text and a question. 
Answer the question based on the information given in the text

## Text ##
{context}

## Conversation ##
{conversation}
\n
Question : {question}
Answer: 

"""

prompt = PromptTemplate.from_template(template)

def chat(question): 
    
    global chatHistory
    global latestPrompt
    # Retrieve docs from vector DB
    #docs = vectors.similarity_search(question, k = 4)
    #contextString = combineDocs2(docs)
    contextString = retriever.invoke(question)
    
    query = prompt.format(conversation = chatHistory, context = contextString, question = question)
    latestPrompt = query
    response = model.invoke(query)
    
    chatHistory = chatHistory + "\nQuestion : " + question + "\nAnswer : " + response

    return response


In [16]:
chat("what are the various parts of the printer?")

'According to the text, the various parts of the HP DeskJet 3700 All-in-One series printer are:\n\n1. Paper-width guide\n2. Input tray extender\n3. Input tray\n4. Input tray shield\n5. Scanner\n6. Scan path\n7. Scan light\n8. Control panel\n9. Cartridge access door\n10. Cartridges\n11. Output tray\n12. Output tray extender (also referred to as the tray extender)\n13. Paper catch\n14. Power connection\n15. USB port\n\nThese parts are listed on page 7 of the user guide.'

In [17]:
print(chatHistory)



Question : what are the various parts of the printer?
Answer : According to the text, the various parts of the HP DeskJet 3700 All-in-One series printer are:

1. Paper-width guide
2. Input tray extender
3. Input tray
4. Input tray shield
5. Scanner
6. Scan path
7. Scan light
8. Control panel
9. Cartridge access door
10. Cartridges
11. Output tray
12. Output tray extender (also referred to as the tray extender)
13. Paper catch
14. Power connection
15. USB port

These parts are listed on page 7 of the user guide.


In [18]:
chat("Can you give the previous answer in a paragraph format?")

'According to the provided text, which appears to be a user guide for the HP DeskJet 3700 All-in-One series printer, the various parts of this printer include the paper-width guide, input tray extender, input tray, input tray shield, scanner, scan path, scan light, control panel, cartridge access door, cartridges, output tray, and output tray extender (also referred to as the tray extender). Additionally, there is a paper catch and power connection.'

In [19]:
chat("which paper options do i have for everyday printing?")

'According to the provided text, you can use plain papers with the ColorLok logo for printing and copying of everyday documents. These papers are independently tested to meet high standards of reliability and print quality, producing documents with crisp, vivid color, bolder blacks, and shorter dry-time than ordinary plain papers.'

In [20]:
print(chatHistory)



Question : what are the various parts of the printer?
Answer : According to the text, the various parts of the HP DeskJet 3700 All-in-One series printer are:

1. Paper-width guide
2. Input tray extender
3. Input tray
4. Input tray shield
5. Scanner
6. Scan path
7. Scan light
8. Control panel
9. Cartridge access door
10. Cartridges
11. Output tray
12. Output tray extender (also referred to as the tray extender)
13. Paper catch
14. Power connection
15. USB port

These parts are listed on page 7 of the user guide.
Question : Can you give the previous answer in a paragraph format?
Answer : According to the provided text, which appears to be a user guide for the HP DeskJet 3700 All-in-One series printer, the various parts of this printer include the paper-width guide, input tray extender, input tray, input tray shield, scanner, scan path, scan light, control panel, cartridge access door, cartridges, output tray, and output tray extender (also referred to as the tray extender). Additionall

In [21]:
print(chat("How can I order hpe supplies?"))

Based on the provided text, which appears to be a user guide for the HP DeskJet 3700 All-in-One series printer, you can order HPE (HP) supplies by visiting their website at www.hp.com/go/anticounterfeit. This is mentioned in the text as "If you believe the purchased original HP cartridges, go to: www.hp.com/go/anticounterfeit".


In [22]:
print(latestPrompt)


You are a helpful assistant. You are given some text and a question. 
Answer the question based on the information given in the text

## Text ##
[Document(metadata={'source': 'onePrinterBook\\HP - Deskjet-3700-Printer.pdf', 'file_path': 'onePrinterBook\\HP - Deskjet-3700-Printer.pdf', 'page': 52, 'total_pages': 118, 'format': 'PDF 1.6', 'title': 'HP DeskJet 3700 All-in-One series User Guide – ENWW', 'author': 'HP IWS Design Customer Education', 'subject': '', 'keywords': 'Edition 1, 9/2016', 'creator': 'AH XSL Formatter V6.2 MR6 for Windows (x64) : 6.2.8.19325 (2015/01/06 12:40JST)', 'producer': 'Antenna House PDF Output Library 6.2.640 (Windows (x64))', 'creationDate': "D:20160524165601+08'00'", 'modDate': "D:20160525102417+08'00'", 'trapped': ''}, page_content='Order ink supplies\nBefore ordering cartridges, locate the correct cartridge number.\nTo locate the cartridge number on the printer\n●\nThe cartridge number is located inside the cartridge access door.\nTo locate the cartridg

In [23]:
print(chat("How can I load full size paper"))

According to the provided text, to load full-size paper on the HP DeskJet 3700 All-in-One series printer, follow these steps:

1. Raise the input tray and input tray extender.
2. Insert a stack of paper into the input tray with the short edge down and the print side up, and slide the stack of paper down until it stops.
3. Slide the paper-width guide to the right until it stops at the edge of the paper.

Please note that this information is based on page 13 of the user guide.


In [24]:
print(chat("Can you give me the previous answer in paragraph form?"))

Here is the answer in paragraph form:

To load full-size paper on your HP DeskJet 3700 All-in-One series printer, start by raising the input tray and input tray extender. Next, insert a stack of paper into the input tray with the short edge down and the print side up, and slide the stack of paper down until it stops. Finally, slide the paper-width guide to the right until it stops at the edge of the paper. Follow these steps to successfully load full-size paper on your printer.


In [25]:
print(chatHistory)



Question : what are the various parts of the printer?
Answer : According to the text, the various parts of the HP DeskJet 3700 All-in-One series printer are:

1. Paper-width guide
2. Input tray extender
3. Input tray
4. Input tray shield
5. Scanner
6. Scan path
7. Scan light
8. Control panel
9. Cartridge access door
10. Cartridges
11. Output tray
12. Output tray extender (also referred to as the tray extender)
13. Paper catch
14. Power connection
15. USB port

These parts are listed on page 7 of the user guide.
Question : Can you give the previous answer in a paragraph format?
Answer : According to the provided text, which appears to be a user guide for the HP DeskJet 3700 All-in-One series printer, the various parts of this printer include the paper-width guide, input tray extender, input tray, input tray shield, scanner, scan path, scan light, control panel, cartridge access door, cartridges, output tray, and output tray extender (also referred to as the tray extender). Additionall